In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 1. Load data
Churn_Metric = pd.read_csv('Churn_Metric.csv') 
Churn_Metric


In [ ]:
# 2. Features for Clustering (Recency, Frequency, Monetary, and Profit Margin)
features = ['days_since_last_order', 'total_revenue', 'total_profit', 'avg_cycle_days']
X = Churn_Metric[features]
X

In [ ]:
# 3. Scale the data (Crucial for K-Means performance)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

In [ ]:

# 4. K-Means Clustering (Using 4 clusters)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
Churn_Metric['Cluster'] = kmeans.fit_predict(X_scaled)
Churn_Metric


In [ ]:
# 5. Clusters
# Note: Logic below depends on cluster means, usually:
# Cluster 0: High Rev/Low Recency -> "At-Risk Giants"
# Cluster 1: High Rev/High Recency -> "Champions"
# Cluster 2: Low Rev/High Recency -> "Steady Small Accounts"
# Cluster 3: New/Infrequent -> "Spot Buyers"
cluster_map = {0: 'At-Risk Giants', 1: 'Champions', 2: 'Steady Accounts', 3: 'Spot Buyers'}
Churn_Metric['Segment'] = Churn_Metric['Cluster'].map(cluster_map)

Churn_Metric[['Customer_Name', 'Segment', 'total_revenue']]

In [ ]:
#Elbow Method to determine optimal K
# Calculate inertia for different K values
inertia = []
K_range = range(1, 10)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

# Plot the Elbow Curve
plt.figure(figsize=(8, 4))
plt.plot(K_range, inertia, 'bx-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
plt.figure(figsize=(12, 7))
sns.scatterplot(
    data=Churn_Metric, 
    x='days_since_last_order', 
    y='total_revenue', 
    hue='Segment', 
    size='total_profit',  # Bubble size shows profitability
    sizes=(50, 500),
    palette='viridis',
    alpha=0.7
)

# Strategic Thresholds
plt.axvline(Churn_Metric['avg_cycle_days'].mean(), color='red', linestyle='--', label='Avg Industry Cycle')
plt.title('Manufacturing Client Segmentation: Revenue vs. Recency', fontsize=15)
plt.xlabel('Days Since Last Order')
plt.ylabel('Total Revenue ($)')
plt.legend(title='Customer Persona', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Filter for High Risk clients and sort by revenue
top_risk_clients = Churn_Metric[Churn_Metric['churn_status'] == 'High Risk'].sort_values(by='total_revenue', ascending=False).head(10)

plt.figure(figsize=(12, 8))
sns.barplot(
    data=top_risk_clients, 
    x='total_revenue', 
    y='Customer_Name', 
    palette='Reds_r'
)

# Adding the 'Days Overdue' as text on the bars
for i, row in enumerate(top_risk_clients.itertuples()):
    plt.text(row.total_revenue, i, f' {int(row.days_since_last_order)} days overdue', va='center', fontweight='bold')

plt.title('Top 10 High-Value Clients in the Danger Zone', fontsize=16)
plt.xlabel('Revenue at Risk ($)')
plt.ylabel('Customer Name')
plt.show()

In [ ]:
#High-Risk Revenue Impact

risk_summary = Churn_Metric.groupby('churn_status')['total_revenue'].sum().reset_index()

plt.figure(figsize=(10, 6))
colors = ['#2ca02c', '#ff7f0e', '#d62728'] # Green, Orange, Red
sns.barplot(data=risk_summary, x='churn_status', y='total_revenue', palette=colors)

# Add data labels to show dollar amounts
for i, val in enumerate(risk_summary['total_revenue']):
    plt.text(i, val, f'${val:,.0f}', ha='center', va='bottom', fontweight='bold')

plt.title('Total Revenue Exposure by Churn Risk Category', fontsize=14)
plt.ylabel('Total Revenue ($)')
plt.xlabel('Churn Risk Status')
plt.show()

In [ ]:
# 1. Filter for only 'High Risk' status from the $55M category
high_risk_list = Churn_Metric[Churn_Metric['churn_status'] == 'High Risk'].copy()

# 2. Sort by total_revenue to prioritize the biggest losses first
high_risk_list = high_risk_list.sort_values(by='total_revenue', ascending=False)

# 3. Create a 'Priority Score' 
# This combines Revenue and how late they are (Deviation)
high_risk_list['priority_score'] = (
    high_risk_list['total_revenue'] * (high_risk_list['days_since_last_order'] / high_risk_list['avg_cycle_days'])
)

# 4. Select relevant columns for the Sales Team
action_report = high_risk_list[[
    'Customer_Name', 
    'Region', 
    'total_revenue', 
    'avg_cycle_days', 
    'days_since_last_order',
    'last_purchase_date'
]]
action_report=action_report.sort_values(by=['total_revenue', 'days_since_last_order'], ascending=False)
# 5. Export to CSV
action_report.to_csv('High_Risk_Customer_Intervention_List.csv', index=False)